In [ ]:
%matplotlib inline
from scipy.signal import butter, filtfilt
import matplotlib.pyplot as plt
import wfdb as w
import numpy as np
import pandas as pd
record = w.rdrecord('100', pn_dir='mitdb')
annotation = w.rdann('100', "atr", pn_dir = "mitdb")
array = np.array(record.p_signal)
fs = 360

In [ ]:
#filtering the signal

class Prefiltering_QRS():
    def band_pass_filter(self, signal):
        low = 5
        high = 15
        b, a = butter(2, [low/(fs/2), high/(fs/2)], btype = 'bandpass')
        y = filtfilt(b, a, signal)
        return y

    def derivative_filter(self, signal):
        y = np.zeros_like(signal)
        for n in range(4, len(signal)):
            y[n] = (2*signal[n] + signal[n-1] - signal[n-3] - 2*signal[n-4]) * fs/8
        return y
    
    def squaring_filter(self, signal):
        y = signal ** 2
        return y
    
    def moving_window(self, signal):
        y = signal.copy()                       
        N = round(0.150 * annotation.fs)
        sum = 0
        for i in range(N):                     
            sum += signal[i]/N
            y[i] = sum
        for i in range(N, len(y)):
            sum -= signal[i-N]/N
            sum += signal[i]/N
            y[i] = sum
        return y
    
    def solve(self, signal):     
        #Since all methods work with ndarrays, I'll select x derivation and convert it to ndarray
        signal_input = signal.iloc[:, 1].to_numpy()                        
                   
        global bpass
        bpass = self.band_pass_filter(signal_input)       

        global derivative
        derivative = self.derivative_filter(bpass)

        global square
        square = self.squaring_filter(derivative)               

        global mwin
        mwin = self.moving_window(square)

        return mwin


QRS_detector = Prefiltering_QRS()
ecg_array = np.array([list(range(len(record.adc()))), record.adc()[:,0]]).T
ecg = pd.DataFrame(ecg_array, columns= ['timestamp', 'ecg_values'])
output_signal = QRS_detector.solve(ecg)                      

In [ ]:
#plotting signal after bandpass filter
plt.figure(figsize = (20,4), dpi = 100) 
plt.xticks(np.arange(0, len(bpass)+1, 150))
plt.plot(bpass[32:len(bpass)-2])
plt.xlabel("campioni")
plt.ylabel("MLIImV")
plt.title("Segnale con filtro passabanda")

#plotting signal after derivative filter
plt.figure(figsize = (20, 4), dpi = 100)
plt.xticks(np.arange(0, len(derivative)+1, 150))
plt.plot(derivative[32:len(derivative)-2])
plt.xlabel("campioni")
plt.ylabel("MLIImV")
plt.title("Segnale con filtro  derivativo")

#plotting signal after square filter
plt.figure(figsize = (20, 4), dpi = 100)
plt.xticks(np.arange(0,len(square)+1, 150))
plt.plot(square[32:len(square)-2])
plt.xlabel("campioni")
plt.ylabel("MLIImV")
plt.title("Segnale con filtro square")

#plotting signal after moving window
plt.figure(figsize = (20, 4), dpi=100)
plt.xticks(np.arange(0,len(mwin)+1), 150)
plt.plot(mwin[100:len(mwin)-2])
plt.xlabel("campioni")
plt.ylabel("MLIImV")
plt.title("Segnale con moving window")